# Financial Analysis 

This notebook contains :
- An automation of the financial data gathering process using the **Financial Modeling Prep API**.
- An example of the financial analysis of **....** through a Waterfall Chart.

### Ressources
- Financial Modeling Prep API Documentation : https://financialmodelingprep.com/developer/docs/ 
- Plotly financial charts : https://plotly.com/python/financial-charts/

## 1 Data Gathering and Preparation

### 1.1 Loading modules

In [327]:
import numpy as np  # Scientific computing
import pandas as pd # Data analysis and manipulation

from datetime import datetime # Dates and times manipulations

import requests # Send HTTP requests easily

import plotly.graph_objects as go # Graphing library for interactive, publication-quality graphs
                                  # pip install plotly==4.5.4

## 2 Gathering the financial data using the Financial Modeling Prep API

In [329]:
# Creating a function to get the financial data by specifying the Ticker and the Company Valuation
def get_financials(ticker, company_valuation):
    
    # Sending an HTTP request to the Financial Modeling Prep API and convert it to a json() file 
    resp = requests.get('https://financialmodelingprep.com/api/v3/financials/{}/{}?period=quarter'.format(company_valuation, ticker)).json()
    
    # Extracting the financials (Dropping the symbol key)
    financials = resp['financials']
    
    # Setting up the diplay format of the floats in the pandas dataframes
    pd.options.display.float_format = '{:,}'.format
    
    # Creating the data frame using pandas from_dict() function
    df = pd.DataFrame.from_dict(financials)
    
    # Converting the 'date' column to a datetime type using datetime
    df['date'] = pd.to_datetime(df['date'])

    # Setting the index to 'date'
    df.set_index('date', inplace = True)

    # Converting the columns into Numeric values
    df = df.apply(pd.to_numeric, errors = 'coerce')
    
    return df

In [331]:
ticker = 'AAPL'
company_valuation = 'income-statement'

df = get_financials(ticker, company_valuation)
df.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
date,,,,,,,,,,,,,,,,,,,,,
2019-12-28,"91,819,000,000.0",0.433775765147,"56,602,000,000.0","35,217,000,000.0","4,451,000,000.0","5,197,000,000.0","9,648,000,000.0","25,569,000,000.0","785,000,000.0","25,918,000,000.0",...,0.38354806739299996,0.309140809636,0.27847177599399997,0.24217209945700002,0.309402193446,"28,385,000,000.0","25,569,000,000.0","22,236,000,000.0",0.282272732223,0.24217209945700002
2019-09-28,"64,040,000,000.0",0.190135479195,"39,727,000,000.0","24,313,000,000.0","4,110,000,000.0","4,578,000,000.0","8,688,000,000.0","15,625,000,000.0","810,000,000.0","16,127,000,000.0",...,0.379653341661,0.293628981886,0.24398813241700001,0.213710181137,0.267535915053,"18,804,000,000.0","15,625,000,000.0","13,686,000,000.0",0.251826983136,0.213710181137
2019-06-29,"53,809,000,000.0",-0.0724984917694,"33,582,000,000.0","20,227,000,000.0","4,257,000,000.0","4,426,000,000.0","8,683,000,000.0","11,544,000,000.0","866,000,000.0","11,911,000,000.0",...,0.375903659239,0.269044211935,0.21453660168400002,0.186660224126,0.17907784943,"14,477,000,000.0","11,544,000,000.0","10,044,000,000.0",0.22135702206,0.186660224126
2019-03-30,"58,015,000,000.0",-0.311884711185,"36,194,000,000.0","21,821,000,000.0","3,948,000,000.0","4,458,000,000.0","8,406,000,000.0","13,415,000,000.0","1,010,000,000.0","13,793,000,000.0",...,0.376126863742,0.283633543049,0.231233301732,0.199276049298,0.151547013703,"16,455,000,000.0","13,415,000,000.0","11,561,000,000.0",0.23774885805399998,0.199276049298
2018-12-29,"84,310,000,000.0",0.340381558029,"52,279,000,000.0","32,031,000,000.0","3,902,000,000.0","4,783,000,000.0","8,685,000,000.0","23,346,000,000.0","890,000,000.0","23,906,000,000.0",...,0.379919345273,0.31717471237100003,0.27690665401500003,0.236804649508,0.276776183134,"26,741,000,000.0","23,346,000,000.0","19,965,000,000.0",0.283548807971,0.236804649508


## Plotting the results

### Set-up

In [332]:
# Creating the list of columns that we will display
columns = ["Revenue", "Cost of Revenue", "Gross Profit", "R&D Expenses", "SG&A Expense", "Operating Expenses", 'Interest Expense', 'Earnings before Tax', 'Income Tax Expense', 'Net Income']

# Getting the first value from each column
values = [df[column].values[0] for column in columns]

# Transforming the costs, expenses and taxes into their negative values
values = [- value if count in [1, 3, 4, 6, 8] else value for (count, value) in enumerate(values)]

# Creating the text list of each column for the waterfall chart
text = [f'{value / 10**9} B' for value in values] 

# Setting up the measures
measures = ["relative", "relative", "total", "relative", "relative", "total", "realtive", "total", "relative", "total"]

In [335]:
# Creating the waterfall chart of the last quarter profit and loss statement
fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = measures,
    x = columns,
    textposition = "outside",
    text = text ,
    y = values,
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Profit and Loss statement for Last Quarter",
        showlegend = True,
        yaxis_title="USD ($)",
)

fig.show()